<a href="https://colab.research.google.com/github/restrepo/dask_bug/blob/master/dfdask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dask bug report
Real json data from http://lens.org. The original data with exactly the same `ValueError` to be shown below, is for some specific set of 50,000 records in a file of 150MB. Other similar sets do not show the error and load smootly. Here one small extract of 191 records displaying exactly the same error. 

The error appears when trying to load a json file into a Dask DataFrame with the `blocksize` option with some specific number of records, in this example: 191 records. The error seem to not depend on some specific record, e.g the first one or the last one, or any subset as displayed below:

In [0]:
import dask.dataframe as dd

In [0]:
!wget https://github.com/restrepo/dask_bug/raw/master/data.json 2> /dev/nul

Not problem without `blocksize`

In [5]:
ln=dd.read_json('data.json')
ln.compute().shape

(191, 23)

Force `ValueError` with first record and any `blocksize`

In [0]:
%%bash
tail -n191 data.json > bad.json
tail -n190 data.json > good.json

In [7]:
ln=dd.read_json('good.json',blocksize=50e6)
ln.compute().shape

(190, 23)

In [8]:
ln=dd.read_json('bad.json',blocksize=50e6)
ln.compute().shape

(191, 23)

Force `ValueError` with last record and any `blocksize`

In [0]:
%%bash
head -n191 data.json > bad.json
head -n190 data.json > good.json

In [10]:
ln=dd.read_json('good.json',blocksize=50e6)
ln.compute().shape

(190, 23)

In [11]:
ln=dd.read_json('bad.json',blocksize=50e6)
ln.compute().shape

(191, 23)

Any partial data set with less than 191 records loads properly:

In [0]:
! head -n120 data.json| tail -n10  > good.json

In [13]:
ln=dd.read_json('good.json',blocksize=50e6)
ln.compute().shape

(10, 23)